Neural network can be contructed by using *torch.nn* package

An *nn.Module* contains layers, and a method *forward(input)* that return the *output*

## Define the network


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 1 input image channels (gray scale image), 6 outputs channels, 3x3 conv
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation; y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120) # Input: 16 channels, 6 width, 6 heights
                                          # Output: 120 hidden units (120 activation values)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 numbers
    
    def forward(self, x):
        # Max pooling over a 2x2 window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the slice window is square, using specific single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions expect the batch dimension
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


The learnable parameters of a model are returned by net.parameters()

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Conv1 weight

10
torch.Size([6, 1, 3, 3])


Let try a random input 32x32

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0289,  0.0110, -0.0476, -0.1319, -0.0399,  0.0412,  0.0308, -0.0412,
         -0.1105,  0.0248]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## Note
* *torch.nn* only support mini-batches. The entire *torch.nn* package only supports inputs that are a mini-batch of samples, and not a package

* For example, *nn.Conv2D* will take 4D of Tensor *nSample x nChannels x Height x Width*

* If you have a single sample, using *input.unsqueeze(0) to add a fake batch dimension

## Recap
* *torch.Tensor* - A multi-dimensional array with support for autograd operation like *backward()*
* nn.Module - Neural network module
* nn.Parameter - A kind of Tensor, that is *automatically registered* as a parameter when assigned as an attribute to a *Module*
* autograd.Function - Implements forward and backward definitions of an autograd operation. Every *Tensor* operation creates at least a single *Function* node that connects to functions that created a *Tensor* and encodes its history.

## Loss function

In [7]:
output = net(input)
target = torch.randn(10) # a dummy target for example
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4810, grad_fn=<MseLossBackward>)


## Backprop

You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0109,  0.0035,  0.0008, -0.0066, -0.0007, -0.0018])


## Update the weights with torch.optim

In [9]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

### Note

Observe how gradient buffers had to be manually set to zero using *optimizer.zero_grad()*. This is because gradients are accumulated to existing gradients, so needing to clear the before existing gradients